In [39]:
import splink.comparison_library as cl
import pandas as pd
import numpy as np
import os, sys
from splink.exploratory import completeness_chart
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets
from splink_tools import *
from pprint import pprint 

def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")

Current working directory:  /home/ubuntu/OrgSync


In [40]:
base_path = os.path.join("data", "splink")

data_path = os.path.join(base_path, "all_data.json")
postcodes_path = os.path.join(base_path, "parsed_postcodes.json")

# load the data
df = pd.read_json(data_path)
df_postcodes = pd.read_json(postcodes_path)
print(f"len(df): {len(df)}")
print(f"len(df_postcodes): {len(df_postcodes)}")
print(f"unique_id in df: {df['unique_id'].nunique()}")
print(f"unique_id in df_postcodes: {df_postcodes['unique_id'].nunique()}")

data_unique_ids = df['unique_id'].to_list()
postcodes_unique_ids = df_postcodes['unique_id'].to_list()
data_datasets = df["dataset"].to_list()
postcodes_datasets = df_postcodes["dataset"].to_list()
# check if the unique_ids in the data and postcodes are the same
assert data_unique_ids == postcodes_unique_ids
assert data_datasets == postcodes_datasets

df.head()

len(df): 105573
len(df_postcodes): 105573
unique_id in df: 74508
unique_id in df_postcodes: 74508


,unique_id,name,postcode,dataset
0,1906596,nuclear decommissioning authority nda,ca24 3hu,cordis
1,2123954,south west tourism limited,ex2 5wt,cordis
2,1915686,welsh government,cf10 3nq,cordis
3,2128407,terrasalus limited,le15 9el,cordis
4,2168344,ol pharma partners ltd,sg4 7dp,cordis


# ISSUE
Duplicate unique_ids in the dataset... Where have these come from??

In [41]:
df_postcodes.head()

,unique_id,dataset,parsed.postcode,parsed.original,parsed.incode,parsed.outcode,parsed.area,parsed.district,parsed.sub_district,parsed.sector,parsed.unit,parsed.fix_distance,parsed.is_in_ons_postcode_directory
0,1906596,cordis,CA24 3HU,ca24 3hu,3HU,CA24,CA,CA24,None,CA24 3,HU,0.0,1.0
1,2123954,cordis,EX2 5WT,ex2 5wt,5WT,EX2,EX,EX2,None,EX2 5,WT,0.0,0.0
2,1915686,cordis,CF10 3NQ,cf10 3nq,3NQ,CF10,CF,CF10,None,CF10 3,NQ,0.0,1.0
3,2128407,cordis,LE15 9EL,le15 9el,9EL,LE15,LE,LE15,None,LE15 9,EL,0.0,1.0
4,2168344,cordis,SG4 7DP,sg4 7dp,7DP,SG4,SG,SG4,None,SG4 7,DP,0.0,1.0


In [42]:
def pd_create_parsed_postcode_columns(data, postcode_data):
    """
    Marge data dn post_code on unique_id

    Inputs:
    data: pd.DataFrame
    postcode_data: pd.DataFrame
    """
    merged_data = pd.merge(data, postcode_data, on=["unique_id", "dataset"], how="outer", suffixes=("", "_postcode"))
    # data = pd.concat([data, postcode_data], axis=1)
    return merged_data

df = pd_create_parsed_postcode_columns(df, df_postcodes)
df.head()

,unique_id,name,postcode,dataset,parsed.postcode,parsed.original,parsed.incode,parsed.outcode,parsed.area,parsed.district,parsed.sub_district,parsed.sector,parsed.unit,parsed.fix_distance,parsed.is_in_ons_postcode_directory
0,1614374,inlecom systems ltd,None,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
1,1614383,london metropolitan university global policy i...,None,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
2,1614807,cpl scientific publishing services ltd,None,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
3,1653759,alkane energy uk limited,None,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN
4,1653780,smart moves limited ta city car club,None,cordis,None,None,None,None,None,None,None,None,None,NaN,NaN


In [43]:
len(df)

14023521

In [44]:
db_api = DuckDBAPI()
# Conver empty lists to null before running...#
completeness_chart(gtr_persons, db_api=db_api, table_names_for_chart=["gtr_persons"])


NameError: name 'gtr_persons' is not defined